In [ ]:
import avstack  # need to import to set the registriesc
import avapi  # need to import to set the registries
import numpy as np

In [ ]:
from avapi.carla import CarlaScenesManager
from avapi.kitti import KittiScenesManager
from avapi.nuscenes import nuScenesManager

dataset = "kitti"

if dataset == "nuscenes":
    nusc_data_dir = "../data/nuScenes"
    SM = nuScenesManager(nusc_data_dir)
    SD = SM.get_scene_dataset_by_name("scene-0103")
    ds_name = "nuscenes"
    agent_name = None
    cam_name = "main_camera"
    lid_name = "main_lidar"
elif dataset == "kitti":
    kitti_obj_data_dir = "../data/KITTI/object/"
    kitti_raw_data_dir = "../data/KITTI/raw"
    SM = KittiScenesManager(kitti_obj_data_dir, kitti_raw_data_dir)
    SD = SM.get_scene_dataset_by_index(0)
    ds_name = "kitti"
    agent_name = None
    cam_name = "main_camera"
    lid_name = "main_lidar"
elif dataset == "carla":
    carla_data_dir = "../data/CARLA/multi-agent-v1/"
    SM = CarlaScenesManager(data_dir=carla_data_dir)
    SD = SM.get_scene_dataset_by_index(1)
    ds_name = "carla-vehicle"
    agent_name = 0
    cam_name = "camera-0"
    lid_name = "lidar-0"
else:
    raise NotImplementedError(dataset)


# Test Perception and Object Tracking


#### Run perception and tracking

In [ ]:
from avstack.geometry import GlobalOrigin3D
from avstack.modules.perception.object3d import MMDetObjectDetector3D
from avstack.modules.tracking.tracker3d import BasicBoxTracker3D

# Load models
perception = MMDetObjectDetector3D(model="pointpillars", dataset=ds_name)
tracker = BasicBoxTracker3D(check_reference=False)

# look over all frames from a scene
first_frame = 1
last_frame = 20
all_imgs = []
all_times = []
all_truths = []
all_dets = []
all_tracks = []
frames = SD.get_frames(sensor=cam_name, agent=agent_name)
for frame in frames[first_frame:last_frame]:
    # get data
    ts = SD.get_timestamp(frame)
    all_times.append(ts)
    img = SD.get_image(frame=frame, sensor=cam_name, agent=agent_name)
    all_imgs.append(img)
    pc = SD.get_lidar(frame=frame, sensor=lid_name, agent=agent_name)
    gt_objects_global = SD.get_objects(frame, sensor=lid_name, agent=agent_name, in_global=True)
    all_truths.append(gt_objects_global)

    # run inference
    dets_local = perception(pc)
    dets_global = dets_local.apply_and_return("change_reference", GlobalOrigin3D, inplace=False)
    all_dets.append(dets_global)
    tracks_global = tracker(dets_global, platform=GlobalOrigin3D)
    all_tracks.append(tracks_global)

#### Evaluate performance

In [ ]:
from avapi.evaluation.ospa import OspaMetric
from avstack.modules.assignment import gnn_single_frame_assign

# parameters for evaluation
assign_radius = 2.0
n_tru = []
n_trk = []
n_asg = []
ospas = []

# run evaluation for every frame
for truths, tracks in zip(all_truths, all_tracks):
    x_truths = [tru.position.x for tru in truths]
    x_tracks = [trk.position.x for trk in tracks]
    A = np.array(
        [[np.linalg.norm(l1 - l2) for l1 in x_tracks] for l2 in x_truths]
    )
    assign = gnn_single_frame_assign(A, cost_threshold=assign_radius)
    n_tru.append(len(x_truths))
    n_trk.append(len(x_tracks))
    n_asg.append(len(assign))
    ospas.append(OspaMetric.cost(tracks=x_tracks, truths=x_truths, c=1))

In [ ]:
import matplotlib.pyplot as plt
from avapi.visualize.snapshot import show_image_with_boxes

# ===========================================
# show a frame
# ===========================================
frame = 18
show_image_with_boxes(all_imgs[frame], all_truths[frame], inline=True)
show_image_with_boxes(all_imgs[frame], all_dets[frame], inline=True)
show_image_with_boxes(all_imgs[frame], all_tracks[frame], inline=True)

# ===========================================
# show metrics
# ===========================================

fig, axs = plt.subplots(nrows=2, ncols=1, figsize=(6,10))

# first: assignments
axs[0].plot(all_times, n_tru, marker="o", linestyle="--", label="Num Truths")
axs[0].plot(all_times, n_trk, marker="o", linestyle="--", label="Num Tracks")
axs[0].plot(all_times, n_asg, marker="o", linestyle="--", label="Num Assignments")
axs[0].legend()
axs[0].set_xlabel("Time (s)")
axs[0].set_ylabel("Metric")
axs[0].set_title("Number of Truths, Tracks, and Assignments")

# second: OSPA
axs[1].plot(all_times, ospas, marker="o", linestyle="--")
axs[1].set_xlabel("Time (s)")
axs[1].set_ylabel("Metric")
axs[1].set_title("Optimal Subpattern Assignment (lower = better)")

plt.tight_layout()
plt.show()